In [1]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime
from crawlers.models import *
from crawlers.finlab.pioneers import *
from crawlers.finlab.import_tools import *
import time

In [2]:
class CrawlBrokerTradeTW:
    
    def __init__(self,start_date):
        self.start_date = start_date
        self.start_date_str = start_date.strftime("%Y-%m-%d")
        self.target_name = "台股分點進出資訊"
        self.format = "time_series"
        
    
    def check_trade_day(self):
        stock_range=CrawlStockPriceTW(self.start_date)
        try:
            df=pd.concat([stock_range.crawl_sii(),stock_range.crawl_otc()])['stock_id'].values
            return df
        except ValueError:
            return None
        
    def broker_trade(self,stock_id):
        print(stock_id)
        url='https://fubon-ebrokerdj.fbs.com.tw/z/zc/zco/zco.djhtm?a='+stock_id+'&e='+self.start_date_str+'&f='+self.start_date_str
        r = requests.post(url)
        html_df = pd.read_html(StringIO(r.text))
        df = pd.DataFrame(html_df[2])
        #holiday
        if len(df)<9:
            return  None
        df.columns=df.iloc[5]
        buy_net_avg_cost=float(df[df['買超券商']=='平均買超成本']['買超'].values[0])
        sell_net_avg_cost=float(df[df['賣超券商']=='平均賣超成本']['賣超'].values[0])
        df=df.iloc[6:-3]
        buy_side=df.iloc[:,:5]
        buy_side=buy_side.rename(columns={'買超券商':'broker_name','買進':'buy_num',
                                          '賣出':'sell_num','買超':'net_bs',
                                          '佔成交比重':'transactions_pt'})

        sell_side=df.iloc[:,5:]
        sell_side=sell_side.rename(columns={'賣超券商':'broker_name','買進':'buy_num',
                                            '賣出':'sell_num','賣超':'net_bs',
                                            '佔成交比重':'transactions_pt'})

        df_all=pd.concat([buy_side,sell_side],sort=False).dropna()
        df_all.iloc[:,1:]=df_all.iloc[:,1:].apply(lambda s: pd.to_numeric(s.str.replace('%',''), errors="coerce"))
        df_all['net_bs']=df_all['buy_num']-df_all['sell_num']
        df_all['net_bs_cost']=[i*buy_net_avg_cost if i>0 else i*sell_net_avg_cost for i in df_all['net_bs']]
        df_all['date'] = pd.to_datetime(self.start_date).date()
        df_all['broker_name']=df_all['broker_name'].apply(lambda s:s.replace('證券','')).apply(lambda s:s.replace('(牛牛牛)','犇'))
        df_all['broker_name']=df_all['broker_name'].apply(lambda s:s if '停' not in s else s[:s.index('-')])
        df_all['stock_id']=str(stock_id)
        AddToSQL.add_to_sql(BrokerTradeTW, df_all,['stock_id','date','broker_name'], [{'broker_name':'broker_name'},{'stock_id':'stock_id'}])
        
        return  df_all

    def crawl_main(self):
        crawl_list=self.check_trade_day()
        if crawl_list is not None:
            for stock_id in crawl_list[:5]:
                 self.broker_trade(stock_id)
        else:
            pass



# a=CrawlBrokerTradeTW(datetime.datetime(2020,4,23))
# a.broker_trade('00774B')
# df=broker_trade(stock_id,datetime.datetime(2020,4,23),end_time=None)
# df

a=CrawlBrokerTradeTW(datetime.datetime(2018,1,3))
df=a.broker_trade('6263')
df
# a.crawl_main()

6263
Finish <class 'crawlers.models.BrokerTradeTW'>date:2018-01-03 bulk_create:30
Finish <class 'crawlers.models.BrokerTradeTW'>date:2018-01-03 bulk_update:0


/Users/benbilly3/Finlab_Django/venv/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


5,broker_name,buy_num,sell_num,net_bs,transactions_pt,net_bs_cost,date,stock_id
6,美商高盛,22,0,22,14.86,1362.24,2018-01-03,6263
7,永豐金-板新,16,0,16,10.81,990.72,2018-01-03,6263
8,元大-新莊,11,0,11,7.43,681.12,2018-01-03,6263
9,富邦-天母,10,0,10,6.76,619.20,2018-01-03,6263
10,元大-忠孝,10,0,10,6.76,619.20,2018-01-03,6263
11,元大-新盛,10,1,9,6.08,557.28,2018-01-03,6263
12,群益金鼎-彰化,6,0,6,4.05,371.52,2018-01-03,6263
13,凱基,6,0,6,4.05,371.52,2018-01-03,6263
14,富邦-永和,5,0,5,3.38,309.60,2018-01-03,6263
15,凱基-東門,4,0,4,2.7,247.68,2018-01-03,6263


In [ ]:
BrokerInfoTW.objects.filter(broker_name='臺銀')